In [192]:
import pyodbc
import pandas as pd
import json
import requests
import re
from os import listdir
from os.path import isfile, join

import glob
import os
#for OSX
import jaydebeapi
import subprocess
from jinja2 import Template
import pandas as pd
from pathlib import Path
import numpy as np

In [193]:
db_path = "../CITS20220208AH.mdb"

#/Users/joaoalmeida
ucanaccess_jars = [
    "/Users/joaoalmeida/Downloads/UCanAccess-5.0.1.bin/ucanaccess-5.0.1.jar",
    "/Users/joaoalmeida/Downloads/UCanAccess-5.0.1.bin/lib/commons-lang3-3.8.1.jar",
    "/Users/joaoalmeida/Downloads/UCanAccess-5.0.1.bin/lib/commons-logging-1.2.jar",
    "/Users/joaoalmeida/Downloads/UCanAccess-5.0.1.bin/lib/hsqldb-2.5.0.jar",
    "/Users/joaoalmeida/Downloads/UCanAccess-5.0.1.bin/lib/jackcess-3.0.1.jar",
]
classpath = ":".join(ucanaccess_jars)
cnxn = jaydebeapi.connect(
    "net.ucanaccess.jdbc.UcanaccessDriver",
    f"jdbc:ucanaccess://{db_path};newDatabaseVersion=V2010",
    ["", ""],
    classpath
    )
crsr = cnxn.cursor()

## Selecting Drugs

In [194]:
dd="""
               select e.*,m.Dosagem,m.Nome,m.ind_Generico,ff.descr AS form_farm_descr,ff.Forma_Farm_ID,te.DESCR as tipo_emb_descr,
                tp.DESCR as Tipo_Prod_descr, gh.Descr as Grupo_Hom_descr, ea.Descr as Estado_AIM_Descr, rcdp.Descr as Class_Disp_descr,
                ta.Descr as Titular_AIM_descr, 
                dci.descr AS DCIPT_descr, dci.DCIPT_ID 
               from (((((((((
                   Embalagem as e

                INNER join produto as m on m.prod_id=e.prod_ID)
                left join ref_Tipo_emb as te on te.Tipo_emb_ID=e.Tipo_emb_ID )
                left join ref_Tipo_produto as tp on tp.Tipo_Produto_ID=m.Tipo_Produto_ID)
                left join ref_Forma_Farm as ff on ff.Forma_Farm_ID=m.Forma_Farm_ID)
                left join ref_Grupo_Hom as gh on gh.Grupo_Hom_cod=e.Grupo_Hom_cod)
                left join ref_Estado_AIM as ea on ea.Estado_AIM_ID=m.Estado_AIM_ID)
                left join lnk_prod_CLASS_DISP as cd on cd.prod_id=m.prod_id)
                left join ref_CLASS_DISPENSA_PROD as rcdp on rcdp.class_disp_prod_id=cd.class_disp_prod_id)
                left join ref_Titular_AIM as ta on ta.Titular_AIM_ID=m.Titular_AIM_ID)
                left join ref_DCIPT as dci on dci.DCIPT_ID=m.DCIPT_ID
                where ea.Estado_AIM_ID=1 """

dd_ = pd.read_sql(dd,cnxn)
list_meds=dd_[~pd.isna(dd_["CNPEM"])]["DCIPT_ID"].unique()

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [195]:
list_meds=[str(int(float(k))) for k in list_meds]


In [196]:
produto_sql="""select * from REF_DCIPT where IND_ATIVO='S' and DCIPT_ID in ({})""".format(",".join(list(list_meds)))
med_ids = pd.read_sql(produto_sql,cnxn)
med_ids

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DCIPT_ID,DESCR,ABREV_CHNM,IND_ATIVO
0,1,Acetilsalicilato de lisina,None,S
1,2,Aciclovir,None,S
2,3,Alopurinol,None,S
3,4,Alprazolam,None,S
4,5,Amicacina,None,S
...,...,...,...,...
1616,4059,Casirivimab + Imdevimab,None,S
1617,4063,Vosoritida,None,S
1618,4064,Regdanvimab,None,S
1619,4065,Fumarato de diroximel,None,S


In [197]:
dcipt_ids=med_ids["DCIPT_ID"].values.tolist()
dcipt_ids=[str(d) for d in dcipt_ids]

In [198]:
package_medicinal_product_sql="""
               select e.*,m.Dosagem,m.Nome,m.ind_Generico,ff.descr AS form_farm_descr,ff.Forma_Farm_ID,te.DESCR as tipo_emb_descr,
                tp.DESCR as Tipo_Prod_descr, gh.Descr as Grupo_Hom_descr, ea.Descr as Estado_AIM_Descr, rcdp.Descr as Class_Disp_descr,
                ta.Descr as Titular_AIM_descr, 
                dci.descr AS DCIPT_descr, dci.DCIPT_ID 
               from (((((((((
                   Embalagem as e

                INNER join produto as m on m.prod_id=e.prod_ID)
                left join ref_Tipo_emb as te on te.Tipo_emb_ID=e.Tipo_emb_ID )
                left join ref_Tipo_produto as tp on tp.Tipo_Produto_ID=m.Tipo_Produto_ID)
                left join ref_Forma_Farm as ff on ff.Forma_Farm_ID=m.Forma_Farm_ID)
                left join ref_Grupo_Hom as gh on gh.Grupo_Hom_cod=e.Grupo_Hom_cod)
                left join ref_Estado_AIM as ea on ea.Estado_AIM_ID=m.Estado_AIM_ID)
                left join lnk_prod_CLASS_DISP as cd on cd.prod_id=m.prod_id)
                left join ref_CLASS_DISPENSA_PROD as rcdp on rcdp.class_disp_prod_id=cd.class_disp_prod_id)
                left join ref_Titular_AIM as ta on ta.Titular_AIM_ID=m.Titular_AIM_ID)
                left join ref_DCIPT as dci on dci.DCIPT_ID=m.DCIPT_ID
                where ea.Estado_AIM_ID=1  and dci.DCIPT_ID in ({})""".format(",".join(dcipt_ids))

package_medicinal_product = pd.read_sql(package_medicinal_product_sql,cnxn)
#package_medicinal_product.to_csv("test_package.csv")
#from this, go downwards
package_medicinal_product

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,EMB_ID,PROD_ID,NR_REGISTO,DESCR,GRUPO_HOM_COD,QUANT_TOT,UNID_QUANT_TOT_ID,QUANTIDADE,UNID_QUANT_ID,TIPO_EMB_ID,...,DESCR,FORMA_FARM_ID,DESCR,DESCR,DESCR,DESCR,DESCR,DESCR,DESCR,DCIPT_ID
0,1,1,9929539,Blister - 20 unidade(s),GH0000,20.0,32,NaN,NaN,NaN,...,Comprimido,271.0,None,Medicamento Uso Humano,None,Autorizado,MNSRM,"Perrigo Portugal, Lda.",Ácido acetilsalicílico,117
1,3,1,9900423,Blister - 40 unidade(s),GH0000,40.0,32,NaN,NaN,NaN,...,Comprimido,271.0,None,Medicamento Uso Humano,None,Autorizado,MNSRM,"Perrigo Portugal, Lda.",Ácido acetilsalicílico,117
2,6,3,9935916,Blister - 20 unidade(s),GH0000,20.0,32,NaN,NaN,NaN,...,Cápsula,29.0,None,Medicamento Uso Humano,None,Autorizado,MSRM,Sidefarma - Sociedade Industrial de Expansão F...,Retinol,981
3,7,3,9935924,Blister - 200 unidade(s),GH0000,200.0,32,NaN,NaN,NaN,...,Cápsula,29.0,None,Medicamento Uso Humano,None,Autorizado,MSRM,Sidefarma - Sociedade Industrial de Expansão F...,Retinol,981
4,12,8,9650002,Frasco conta-gotas - 1 unidade(s) - 50 ml,GH0000,1.0,32,50.0,12.0,NaN,...,Solução oral,4.0,None,Medicamento Uso Humano,None,Autorizado,MSRM,Sidefarma - Sociedade Industrial de Expansão F...,Ginkgo biloba,616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46913,2205975,35283,5830625,Frasco para injetáveis - 5 unidade(s),GH0000,5.0,32,NaN,NaN,NaN,...,Pó para solução para perfusão,217.0,None,Medicamento Uso Humano,None,Autorizado,MSRM,"Hikma Farmacêutica (Portugal), S.A.",Amoxicilina + Ácido clavulânico,9
46914,2206095,685184,None,Recipiente unidose - 60 unidade(s),GH0000,60.0,32,NaN,NaN,NaN,...,"Colírio, solução",123.0,None,Medicamento Uso Humano,None,Autorizado,MSRM restrita,Trimb Healthcare AB,Pilocarpina,918
46915,2206215,685284,None,Frasco - 1 unidade(s) - 250 ml,GH0000,1.0,32,250.0,12.0,NaN,...,Suspensão oral,5.0,None,Medicamento Uso Humano,None,Autorizado,MSRM restrita,Dr. Falk Pharma GmbH,Ácido ursodesoxicólico,1184
46916,2206335,685444,None,Ampola - 1 unidade(s) - 10 ml,GH0000,1.0,32,10.0,12.0,NaN,...,Solução injetável,207.0,None,Medicamento Uso Humano,None,Autorizado,MSRM restrita,GE Healthcare BV,Iodeto (131I) de sódio,2207


In [199]:
#get-columns-name
columns_sql="""SELECT * FROM Embalagem limit 5"""
col_name=pd.read_sql(columns_sql,cnxn).columns.tolist()
col_name.extend(["Dosagem","Nome","ind_generico","form_farm_descr","Forma_farm_id","tipo_emb_descr","Tipo_prod_descr","Grupo_hom_descr","Estado_AIM_descr","Class_Disp_descr","Titular_AIM_descr","DCIPT_descr","DCIPT_ID"])
col_name
package_medicinal_product.columns=col_name

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [200]:
def create_package_name(row):
    if None in [row["Dosagem"],row["Nome"],row["form_farm_descr"],row["DESCR"]]:
        print( row["Nome"],row["Dosagem"],row["form_farm_descr"],row["DESCR"],row["EMB_ID"])
        return None
    else:
        return row["Nome"] +" "+row["Dosagem"]+" "+row["form_farm_descr"] + " " + row["DESCR"]

package_medicinal_product["full_name"]=package_medicinal_product.apply(create_package_name,axis=1)


Tagamet 200 mg/ 2ml None Ampola - 10 unidade(s) - 2 ml 2203536


In [201]:
def parse_single_instance(col,tot):
    if col.count("/")==0: #no denominator
    # print(parse_num(col))
        numerator_value,numerator_unit=parse_num(col,tot)
        return numerator_value,numerator_unit,None,None
    elif col.count("/")==1:
        #print(col)
        num=col.split("/")[0]
        dem=col.split("/")[1]
    #     print(num,"   >->",dem)
        numerator_value,numerator_unit=parse_num(num,tot)
        denominator_value,denominator_unit=parse_dem(dem,tot)
        # print(numerator_value,numerator_unit,denominator_value,denominator_unit)
        return numerator_value,numerator_unit,denominator_value,denominator_unit
    else:
        raise ValueError(col,tot)
def parse_num(num,col):
    numerator_value_tag="\d+\.?\d{0,3}"
    numerator_unit_tag="\s?(\D+)$"
    try:
        numerator_value=re.findall(numerator_value_tag,num)[0].strip()
    except:
        #raise ValueError(num,col)
        print(num,col,"Error on numerator value")
        numerator_value=None
    try:
        numerator_unit=str(re.findall(numerator_unit_tag,num)[0].strip())
    except:
        #raise ValueError(num,col)
        print(num,col,"Error on numerator unit")
        numerator_unit=None


    return numerator_value,numerator_unit
def parse_dem(dem,col):
    """
    2 ml
    g 
    0.5 ml
    0.5ml
    ml
    """
    denominator_unit_tag=re.compile("\s?(\D+)$")
    denominator_value_tag=re.compile("\d\.?\d?")
    #print(dem)
    try:
        if denominator_value_tag.match(dem): #if has number
            denominator_value=str(denominator_value_tag.findall(dem)[0].strip())
        else: #defaults to 1
            denominator_value=1
        
        denominator_unit=denominator_unit_tag.findall(dem)[0].strip()

    except:
        raise ValueError(dem)
    return denominator_value,denominator_unit
    # print(denominator_unit)


def parse_strength(row):
  
    """
    cases:
    50mg
    50 mg
    0.5mg/ml
    5mg/10ml
    1 mg/ml
    1200 mg/6 ml + 800 U.I./6 ml
    100 U.I./ml (25% + 75%)
    100 U.I./ml (30% + 70%)
    CCID50 10e6 CCID50
    800mg + 150mg + 200mg + 10mg
    """
  
    col=row["Dosagem"]
    if col in ["Associação","Combinação","associação"]:
        return None,None,None,None
    if col in "CCID50 10e6 CCID50":
        return None,None,None,None #what to do?
   # print(col)
    if col.count("+")==0:

       return parse_single_instance(col,col)
    else:
     #   print(col)
        numerator_unit_list=[]
        denominator_unit_list=[]
        numerator_value_list=[]
        denominator_value_list=[]
        for prod in col.split("+"):
            numerator_value,numerator_unit,denominator_value,denominator_unit=parse_single_instance(prod,col)
            numerator_unit_list.append(numerator_unit)
            if denominator_unit:
                denominator_unit_list.append(denominator_unit)
            
            numerator_value_list.append(numerator_value)
            if denominator_value:
                denominator_value_list.append(denominator_value)
        if len(denominator_value_list)>0:

            return numerator_value_list,numerator_unit_list,denominator_value_list,denominator_unit_list
        else:   
            return numerator_value_list,numerator_unit_list,None,None

package_medicinal_product[["numerator_value","numerator_unit","denominator_value","denominator_unit"]]=package_medicinal_product.apply(parse_strength,axis=1,result_type='expand')


In [202]:
package_medicinal_product[package_medicinal_product["EMB_ID"]==134]["Dosagem"]

37    Associação
Name: Dosagem, dtype: object

In [203]:
package_medicinal_product["full_name"]

0            AAS 500 mg Comprimido Blister - 20 unidade(s)
1            AAS 500 mg Comprimido Blister - 40 unidade(s)
2        A-Vite 50000 U.I. Cápsula Blister - 20 unidade(s)
3        A-Vite 50000 U.I. Cápsula Blister - 200 unidad...
4        Abolibe Forte 40 mg/ml Solução oral Frasco con...
                               ...                        
46913    Amoxicilina + Ácido Clavulânico Hikma 2000+200...
46914    Pilokarpin Trimb  20 mg/ml Colírio, solução Re...
46915    Ursofalk 250 mg/5 ml Suspensão oral Frasco - 1...
46916    Sodium Iodide Injection GE Healthcare 370 MBq/...
46917    Makro-albumon 2 mg Pó para suspensão injetável...
Name: full_name, Length: 46918, dtype: object

In [204]:
package_medicinal_product[["full_name","Dosagem","numerator_value","numerator_unit","denominator_value","denominator_unit"]].iloc[50:90,:]

,full_name,Dosagem,numerator_value,numerator_unit,denominator_value,denominator_unit
50,Adenoscan 3 mg/ml Solução para perfusão Frasco...,3 mg/ml,3,mg,1,ml
51,Adrenalina Labesfal 1 mg/1 ml Solução injetáve...,1 mg/1 ml,1,mg,1,ml
52,Adrenalina Labesfal 1 mg/1 ml Solução injetáve...,1 mg/1 ml,1,mg,1,ml
53,Adrenalina Labesfal 1 mg/1 ml Solução injetáve...,1 mg/1 ml,1,mg,1,ml
54,Adrenalina Labesfal 1 mg/1 ml Solução injetáve...,1 mg/1 ml,1,mg,1,ml
55,Adrenalina Labesfal 2 mg/1 ml Solução injetáve...,2 mg/1 ml,2,mg,1,ml
56,Adrenalina Labesfal 2 mg/1 ml Solução injetáve...,2 mg/1 ml,2,mg,1,ml
57,Adrenalina Labesfal 2 mg/1 ml Solução injetáve...,2 mg/1 ml,2,mg,1,ml
58,Adrenalina Labesfal 2 mg/1 ml Solução injetáve...,2 mg/1 ml,2,mg,1,ml
59,Adrenalina Labesfal 0.1 mg/1 ml Solução injetá...,0.1 mg/1 ml,0.1,mg,1,ml


In [205]:
#800mg + 150mg + 200mg + 10mg
package_medicinal_product[package_medicinal_product["Dosagem"]=="800mg + 150mg + 200mg + 10mg"]


,EMB_ID,PROD_ID,NR_REGISTO,DESCR,GRUPO_HOM_COD,QUANT_TOT,UNID_QUANT_TOT_ID,QUANTIDADE,UNID_QUANT_ID,TIPO_EMB_ID,...,Estado_AIM_descr,Class_Disp_descr,Titular_AIM_descr,DCIPT_descr,DCIPT_ID,full_name,numerator_value,numerator_unit,denominator_value,denominator_unit
41313,2102050,622642,5728373,Frasco - 30 unidade(s),GH0000,30.0,32,NaN,NaN,NaN,...,Autorizado,MSRM restrita - Alínea a),Janssen-Cilag International N.V.,Darunavir + Cobicistate + Emtricitabina + Teno...,3743,Symtuza 800mg + 150mg + 200mg + 10mg Comprimid...,"[800, 150, 200, 10]","[mg, mg, mg, mg]",None,None


In [206]:
def transform_dcipt_to_list(row):
    """ transform combinations to strucutred
    """
    if "+" in row["DCIPT_descr"]:
        result=row["DCIPT_descr"].split("+")
        #print(row["DCIPT_descr"])
        return [prod.strip() for prod in result]
    return row["DCIPT_descr"]
package_medicinal_product["DCIPT_descr_parse"]=package_medicinal_product.apply(transform_dcipt_to_list,axis=1)


In [207]:
package_medicinal_product.head()

,EMB_ID,PROD_ID,NR_REGISTO,DESCR,GRUPO_HOM_COD,QUANT_TOT,UNID_QUANT_TOT_ID,QUANTIDADE,UNID_QUANT_ID,TIPO_EMB_ID,...,Class_Disp_descr,Titular_AIM_descr,DCIPT_descr,DCIPT_ID,full_name,numerator_value,numerator_unit,denominator_value,denominator_unit,DCIPT_descr_parse
0,1,1,9929539,Blister - 20 unidade(s),GH0000,20.0,32,NaN,NaN,NaN,...,MNSRM,"Perrigo Portugal, Lda.",Ácido acetilsalicílico,117,AAS 500 mg Comprimido Blister - 20 unidade(s),500,mg,None,None,Ácido acetilsalicílico
1,3,1,9900423,Blister - 40 unidade(s),GH0000,40.0,32,NaN,NaN,NaN,...,MNSRM,"Perrigo Portugal, Lda.",Ácido acetilsalicílico,117,AAS 500 mg Comprimido Blister - 40 unidade(s),500,mg,None,None,Ácido acetilsalicílico
2,6,3,9935916,Blister - 20 unidade(s),GH0000,20.0,32,NaN,NaN,NaN,...,MSRM,Sidefarma - Sociedade Industrial de Expansão F...,Retinol,981,A-Vite 50000 U.I. Cápsula Blister - 20 unidade(s),50000,U.I.,None,None,Retinol
3,7,3,9935924,Blister - 200 unidade(s),GH0000,200.0,32,NaN,NaN,NaN,...,MSRM,Sidefarma - Sociedade Industrial de Expansão F...,Retinol,981,A-Vite 50000 U.I. Cápsula Blister - 200 unidad...,50000,U.I.,None,None,Retinol
4,12,8,9650002,Frasco conta-gotas - 1 unidade(s) - 50 ml,GH0000,1.0,32,50.0,12.0,NaN,...,MSRM,Sidefarma - Sociedade Industrial de Expansão F...,Ginkgo biloba,616,Abolibe Forte 40 mg/ml Solução oral Frasco con...,40,mg,1,ml,Ginkgo biloba


In [208]:
def transform_list_to_string(x):
    if type(x)==list and len(x)>0:
        x=[str(k) for k in x]
        return ";".join(x)
    return x
package_medicinal_product["numerator_value"]=package_medicinal_product["numerator_value"].apply(transform_list_to_string)
package_medicinal_product["numerator_unit"]=package_medicinal_product["numerator_unit"].apply(transform_list_to_string)
package_medicinal_product["denominator_value"]=package_medicinal_product["denominator_value"].apply(transform_list_to_string)
package_medicinal_product["denominator_unit"]=package_medicinal_product["denominator_unit"].apply(transform_list_to_string)
package_medicinal_product["DCIPT_descr_parse"]=package_medicinal_product["DCIPT_descr_parse"].apply(transform_list_to_string)


In [209]:
routes_sql="""select lpva.PROD_ID,lpva.VIA_ADMIN_ID,rva.DESCR from lnk_prod_via_admin lpva 
left join ref_via_admin rva on lpva.via_admin_id=rva.via_admin_id
  """
routes = pd.read_sql(routes_sql,cnxn)
routes

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,PROD_ID,VIA_ADMIN_ID,DESCR
0,50803,48,Via oral
1,623263,48,Via oral
2,1593,48,Via oral
3,49252,48,Via oral
4,623622,48,Via oral
...,...,...,...
20110,656665,48,Via oral
20111,652044,48,Via oral
20112,652045,48,Via oral
20113,653665,42,Via intravenosa


In [210]:
def concatenate_values_from_list(x):
   # print(type(x))
   # print(x.values)
    #print(";".join([str(d) for d in x.values]))
    return ";".join([str(d) for d in x.values])


In [211]:
# get CPARM and mapp to infarmed
cparm_routes=pd.read_excel("../CPARM_V1.0.xlsx","Routes of Administration")
cparm_routes["mapping_term"]=cparm_routes["Preferred Term (PT) PT | Termo de Preferência PT"]
cparm_routes.loc[cparm_routes["Code EDQM |Código EDQM"]==20020000,"mapping_term"]="Via inalatória"
cparm_routes.loc[cparm_routes["Code EDQM |Código EDQM"]==20047000,"mapping_term"]="Via intravítreo"
cparm_routes.loc[cparm_routes["Code EDQM |Código EDQM"]==20025500,"mapping_term"]="Uso intracameral"
cparm_routes.loc[len(cparm_routes.index)] = cparm_routes[cparm_routes["Code EDQM |Código EDQM"]==20003000].values[0]
cparm_routes.iloc[-1,-1] = "Uso epicutâneo"

In [212]:
mapp_routes_edqm=routes.merge(cparm_routes,how="left",left_on="DESCR",right_on="mapping_term")
mapp_routes_edqm[mapp_routes_edqm["Code EDQM |Código EDQM"].isnull()]
#Uso epicutâneo	


,PROD_ID,VIA_ADMIN_ID,DESCR,Code EDQM |Código EDQM,Fully Specified Name (FSN) EN | Termo Totalmente Especificado EN,Definition EN | Definição EN,Preferred Term (PT) PT | Termo de Preferência PT,Definition PT | Definição PT,Domain | Âmbito,Version | Versão,Type of Change | Tipo Alteração,mapping_term


In [214]:
routes_agg=mapp_routes_edqm.groupby("PROD_ID").aggregate({"VIA_ADMIN_ID":concatenate_values_from_list,"DESCR":concatenate_values_from_list,"Code EDQM |Código EDQM":concatenate_values_from_list,"Fully Specified Name (FSN) EN | Termo Totalmente Especificado EN":concatenate_values_from_list}).reset_index()
routes_agg.columns=["PROD_ID","routes_id","routes_descr","edqm_routes_id","edqm_routes_descr_en"]

In [215]:
product_with_routes=package_medicinal_product.merge(routes_agg,on="PROD_ID",how="left")

In [216]:
atc_sql="""select lpa.PROD_ID,lpa.ATC_COD,fa.DESCR from lnk_prod_atc lpa
left join ref_atc fa on lpa.atc_cod=fa.atc_cod
  """
atc = pd.read_sql(atc_sql,cnxn)
atc

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,PROD_ID,ATC_COD,DESCR
0,600228,N03AX14,levetiracetam
1,52417,N03AX14,levetiracetam
2,52265,N03AX14,levetiracetam
3,54829,J01XA01,vancomycin
4,35943,A16AX05,zinc acetate
...,...,...,...
18910,608082,H01BA04,terlipressin
18911,42315,N05CD08,midazolam
18912,29767,N02BA01,acetylsalicylic acid
18913,584341,N05CD08,midazolam


In [217]:
atc_agg=atc.groupby("PROD_ID").aggregate({"ATC_COD":concatenate_values_from_list,"DESCR":concatenate_values_from_list}).reset_index()
atc_agg.columns=["PROD_ID","atc_code","atc_descr"]

In [218]:
product_with_routes_atc=product_with_routes.merge(atc_agg,on="PROD_ID",how="left")

In [219]:
cparm_pharm_forms=pd.read_excel("../CPARM_V1.0.xlsx","Pharmaceutical Dose Forms")
#remove duplicates terms (tablet for 12200 and 102190000)
cparm_pharm_forms["mapping_term"]=cparm_pharm_forms["Preferred Term (PT) PT | Termo de Preferência PT"]
cparm_pharm_forms["mapping_term"]=cparm_pharm_forms["mapping_term"].apply(lambda x: x.strip())
cparm_pharm_forms.loc[cparm_pharm_forms["Code EDQM |Código EDQM"]==11005000,"mapping_term"]="Solução retal"
cparm_pharm_forms=cparm_pharm_forms[cparm_pharm_forms["Definition EN | Definição EN"]!=" - "]


In [220]:
#for all (NOT DONE)
dose_forms_sql="""select * from ref_Forma_Farm """
dose_forms = pd.read_sql(dose_forms_sql,cnxn)
dose_forms

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,FORMA_FARM_ID,DESCR,ABREV,ABREV_CHNM,IND_ATIVO
0,0,N.A.,None,None,S
1,1,"Gotas orais, solução","Gotas orais, sol.",Sol oral,S
2,2,"Gotas orais, suspensão","Gotas orais, susp.",Susp oral,S
3,3,"Gotas orais, emulsão","Gotas orais, emul.",Emul oral,S
4,4,Solução oral,Sol. oral,Sol oral,S
...,...,...,...,...,...
510,1000,Solução para perfusão em cartucho,None,Sol inj,S
511,1001,Gel oftálmico em recipiente unidose,None,Gel oft,S
512,1002,Película sublingual,None,None,S
513,1003,"Pó, dispersão e veículo para concentrado para ...",None,None,S


In [221]:
selected_dose_forms=product_with_routes_atc[["form_farm_descr","Forma_farm_id"]]
selected_dose_forms.drop_duplicates(inplace=True)
selected_dose_forms

/var/folders/8g/rym67m5d29dcjm2td47sgb_m0000gn/T/ipykernel_2908/1619014294.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_dose_forms.drop_duplicates(inplace=True)


,form_farm_descr,Forma_farm_id
0,Comprimido,271.0
2,Cápsula,29.0
4,Solução oral,4.0
5,Comprimido revestido por película,39.0
17,Pó e solvente para solução injetável,212.0
...,...,...
46150,Dispersão para perfusão,954.0
46475,Pó e solvente para concentrado para solução pa...,536.0
46536,Solução para lavagem oftálmica,129.0
46873,Granulado e veículo para suspensão oral,542.0


In [222]:
mapp_dose_form_edqm=selected_dose_forms.merge(cparm_pharm_forms,how="left",left_on="form_farm_descr",right_on="mapping_term")
mapp_dose_form_edqm[mapp_dose_form_edqm["Code EDQM |Código EDQM"].isnull()]
# Granulado orodispersível
# Sistema transdérmico	
# Solução injetável ou para perfusão

,form_farm_descr,Forma_farm_id,Code EDQM |Código EDQM,Fully Specified Name (FSN) EN | Termo Totalmente Especificado EN,Definition EN | Definição EN,Preferred Term (PT) PT | Termo de Preferência PT,Definition PT | Definição PT,Domain | Âmbito,Version | Versão,Type of Change | Tipo Alteração,mapping_term
49,"Colírio, comprimido e solvente para solução",900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
55,Pó para solução injetável ou para solução para...,530.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
66,Sistema transdérmico,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
68,Solução oral + Pó para solução oral,311.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
79,"Pó para inalação, cápsula",198.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
99,Solução injetável ou para perfusão,548.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
100,Dispersão injetável,348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
109,Granulado para solução oral ou retal,288.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
125,Solvente/Veículo para uso parentérico,220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
126,Creme vaginal + Óvulo,306.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN


In [223]:
cparm_pharm_forms=cparm_pharm_forms[["Code EDQM |Código EDQM","Fully Specified Name (FSN) EN | Termo Totalmente Especificado EN","Preferred Term (PT) PT | Termo de Preferência PT","mapping_term"]]
cparm_pharm_forms.columns=["edqm_dose_form_id","edqm_dose_form_descr_en","edqm_dose_form_descr_pt","mapping_term"]

In [224]:
#add to final dataframe
cparm_pharm_forms
product_with_routes_atc_edqm_dose=product_with_routes_atc.merge(cparm_pharm_forms,how="left",left_on="form_farm_descr",right_on="mapping_term")

In [225]:
product_with_routes_atc_edqm_dose

,EMB_ID,PROD_ID,NR_REGISTO,DESCR,GRUPO_HOM_COD,QUANT_TOT,UNID_QUANT_TOT_ID,QUANTIDADE,UNID_QUANT_ID,TIPO_EMB_ID,...,routes_id,routes_descr,edqm_routes_id,edqm_routes_descr_en,atc_code,atc_descr,edqm_dose_form_id,edqm_dose_form_descr_en,edqm_dose_form_descr_pt,mapping_term
0,1,1,9929539,Blister - 20 unidade(s),GH0000,20.0,32,NaN,NaN,NaN,...,48,Via oral,20053000,Oral use,N02BA01,acetylsalicylic acid,10219000.0,Tablet,Comprimido,Comprimido
1,3,1,9900423,Blister - 40 unidade(s),GH0000,40.0,32,NaN,NaN,NaN,...,48,Via oral,20053000,Oral use,N02BA01,acetylsalicylic acid,10219000.0,Tablet,Comprimido,Comprimido
2,6,3,9935916,Blister - 20 unidade(s),GH0000,20.0,32,NaN,NaN,NaN,...,48,Via oral,20053000,Oral use,A11CA01,retinol (vit A),12100.0,Capsule,Cápsula,Cápsula
3,7,3,9935924,Blister - 200 unidade(s),GH0000,200.0,32,NaN,NaN,NaN,...,48,Via oral,20053000,Oral use,A11CA01,retinol (vit A),12100.0,Capsule,Cápsula,Cápsula
4,12,8,9650002,Frasco conta-gotas - 1 unidade(s) - 50 ml,GH0000,1.0,32,50.0,12.0,NaN,...,48,Via oral,20053000,Oral use,N06DX02,Ginkgo folium,10105000.0,Oral solution,Solução oral,Solução oral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46914,2205975,35283,5830625,Frasco para injetáveis - 5 unidade(s),GH0000,5.0,32,NaN,NaN,NaN,...,42,Via intravenosa,20045000,Intravenous use,J01CR02,amoxicillin and beta-lactamase inhibitor,11212000.0,Powder for solution for infusion,Pó para solução para perfusão,Pó para solução para perfusão
46915,2206095,685184,None,Recipiente unidose - 60 unidade(s),GH0000,60.0,32,NaN,NaN,NaN,...,47,Uso oftálmico,20051000,Ocular use,S01EB01,pilocarpine,10604000.0,"Eye drops, solution","Colírio, solução","Colírio, solução"
46916,2206215,685284,None,Frasco - 1 unidade(s) - 250 ml,GH0000,1.0,32,250.0,12.0,NaN,...,48,Via oral,20053000,Oral use,A05AA02,ursodeoxycholic acid,10106000.0,Oral suspension,Suspensão oral,Suspensão oral
46917,2206335,685444,None,Ampola - 1 unidade(s) - 10 ml,GH0000,1.0,32,10.0,12.0,NaN,...,42,Via intravenosa,20045000,Intravenous use,V09FX02,sodium iodide (123I),11201000.0,Solution for injection,Solução injetável,Solução injetável


In [226]:
#testing with Amlor 10mg cápsulas caixa 100 - nao existe pt
#linha 137 - norvasc 10 mg caixa 30

In [227]:
#check for duplicates
product_with_routes_atc_edqm_dose[product_with_routes_atc_edqm_dose.duplicated(['EMB_ID'], keep=False)]

,EMB_ID,PROD_ID,NR_REGISTO,DESCR,GRUPO_HOM_COD,QUANT_TOT,UNID_QUANT_TOT_ID,QUANTIDADE,UNID_QUANT_ID,TIPO_EMB_ID,...,routes_id,routes_descr,edqm_routes_id,edqm_routes_descr_en,atc_code,atc_descr,edqm_dose_form_id,edqm_dose_form_descr_en,edqm_dose_form_descr_pt,mapping_term
1097,5442,3371,8176339,Ampola - 10 unidade(s) - 10 ml,GH0000,10.0,32,10.00,12.0,NaN,...,42,Via intravenosa,20045000,Intravenous use,N01AH01,fentanyl,11201000.0,Solution for injection,Solução injetável,Solução injetável
1098,5442,3371,8176339,Ampola - 10 unidade(s) - 10 ml,GH0000,10.0,32,10.00,12.0,NaN,...,42,Via intravenosa,20045000,Intravenous use,N01AH01,fentanyl,11201000.0,Solution for injection,Solução injetável,Solução injetável
2283,12204,7363,8640409,Ampola - 5 unidade(s) - 2 ml,GH0000,5.0,32,2.00,12.0,NaN,...,42,Via intravenosa,20045000,Intravenous use,N01AH02,alfentanil,11201000.0,Solution for injection,Solução injetável,Solução injetável
2284,12204,7363,8640409,Ampola - 5 unidade(s) - 2 ml,GH0000,5.0,32,2.00,12.0,NaN,...,42,Via intravenosa,20045000,Intravenous use,N01AH02,alfentanil,11201000.0,Solution for injection,Solução injetável,Solução injetável
2285,12205,7364,8640417,Ampola - 5 unidade(s) - 10 ml,GH0000,5.0,32,10.00,12.0,NaN,...,42,Via intravenosa,20045000,Intravenous use,N01AH02,alfentanil,11201000.0,Solution for injection,Solução injetável,Solução injetável
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45549,2160195,30754,5795141,Ampola - 50 unidade(s) - 10 ml,GH0000,50.0,32,10.00,12.0,NaN,...,42,Via intravenosa,20045000,Intravenous use,N01AH01,fentanyl,11201000.0,Solution for injection,Solução injetável,Solução injetável
45578,2160476,643465,5797204,Saqueta - 1 unidade(s),GH0000,1.0,32,NaN,NaN,NaN,...,48,Via oral,20053000,Oral use,J07AE02,"cholera, live attenuated",13119000.0,Effervescent powder and powder for oral suspen...,Pó efervescente e pó para suspensão oral,Pó efervescente e pó para suspensão oral
45579,2160476,643465,5797204,Saqueta - 1 unidade(s),GH0000,1.0,32,NaN,NaN,NaN,...,48,Via oral,20053000,Oral use,J07AE02,"cholera, live attenuated",13150000.0,Effervescent powder and suspension for oral su...,Pó efervescente e pó para suspensão oral,Pó efervescente e pó para suspensão oral
46569,2182255,668424,5817655,Seringa pré-cheia - 1 unidade(s) - 0.45 ml,GH0000,1.0,32,0.45,12.0,NaN,...,58,Via subcutânea,20066000,Subcutaneous use,N07BC01,buprenorphine,13076000.0,Prolonged-release solution for injection,Solução injetável de libertação prolongada,Solução injetável de libertação prolongada


In [228]:
tt=product_with_routes_atc_edqm_dose["DCIPT_descr_parse"].unique()
list_unparse=[]
for ttt in tt:
    if ";" in ttt:
        for pl in ttt.split(";"):
            list_unparse.append(pl)
    else:
        list_unparse.append(ttt)

In [ ]:
item_dict_parse= {k:v for k,v in zip(list_unparse,list(range(1,len(list_unparse)+1)))}



In [230]:
def mapp_dci_pt_parse(x,map):
    if ";" in x:
        p=[]
        for r in x.split(";"):
            p.append(str(map[r]))
        return ";".join(p)
    else:
        return map[x]

In [231]:
product_with_routes_atc_edqm_dose["DCIPT_id_parse"]=product_with_routes_atc_edqm_dose["DCIPT_descr_parse"].apply(mapp_dci_pt_parse,map=item_dict_parse)

In [232]:
product_with_routes_atc_edqm_dose[product_with_routes_atc_edqm_dose["DCIPT_ID"].isin([174,3446,3600,3633])].to_csv("infarmed_export_v0_amlodipine_multiple.csv")

In [233]:
product_with_routes_atc_edqm_dose.to_csv("infarmed_export_v0.csv")


In [234]:
# multiple elementsaa

with open('template-packaged-medicinal-product R5.fsh', 'r') as file:
    templateString = file.read()
t = Template(templateString,trim_blocks=True)

#DATA_FILE='infarmed_export_v0_amlodipine_multiple.csv'
DATA_FILE='infarmed_export_v0.csv'
df=pd.read_csv(DATA_FILE,index_col=0)
df=df.astype(str)

#Path("../input/fsh/generated").mkdir(parents=True, exist_ok=True)
#t.stream(data=df.iloc[0:2]).dump('input/fsh/be-ampp.fsh')
t.stream(data=df).dump('../../input/fsh/pt-pmp_r5.fsh')

In [235]:
# writing to file
file1 = open('../../input/fsh/pt-pmp_r5.fsh', 'r')
Lines = file1.readlines()
error_count=0
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    if "// ERROR" in line:
        print("{} @{}".format( line.strip(),count))
        error_count+=1

// ERROR[2]  - no ATC for ID:412 @3973
// ERROR[2]  - no ATC for ID:2795 @20712
// ERROR[2]  - no ATC for ID:2796 @20752
// ERROR[2]  - no ATC for ID:2797 @20792
// ERROR[3]  - no EDQM Dose Form for ID:3091 @23049
// ERROR[4]  - no Native Dose Form for ID:3091 @23050
// ERROR[3]  - no EDQM Dose Form for ID:3468 @27035
// ERROR[4]  - no Native Dose Form for ID:3468 @27036
// ERROR[3]  - no EDQM Dose Form for ID:3959 @30575
// ERROR[4]  - no Native Dose Form for ID:3959 @30576
// ERROR[3]  - no EDQM Dose Form for ID:3960 @30608
// ERROR[4]  - no Native Dose Form for ID:3960 @30609
// ERROR[3]  - no EDQM Dose Form for ID:3961 @30641
// ERROR[4]  - no Native Dose Form for ID:3961 @30642
// ERROR[3]  - no EDQM Dose Form for ID:3962 @30674
// ERROR[4]  - no Native Dose Form for ID:3962 @30675
// ERROR[3]  - no EDQM Dose Form for ID:3963 @30707
// ERROR[4]  - no Native Dose Form for ID:3963 @30708
// ERROR[3]  - no EDQM Dose Form for ID:3964 @30740
// ERROR[4]  - no Native Dose Form for ID:39

In [236]:
error_count

2048